In [ ]:
!pip install OpenDartReader

In [ ]:
from financial_data_making import financial_dict
import OpenDartReader
import pandas as pd

# df = pd.read_csv('./merged_code.csv')
call =  financial_dict('4fb4acc1d37b1dae74040e64a9674e018ccfd7fe') # api_key

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str' 
                                                ,'stock_code': 'str'})

df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
print(df.shape)
df.head()
df.to_csv('final_corp_info.csv')

In [ ]:
api_key = '4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)
code, year = '311690', 2020

# 매출액
# 손익계산서
def sales_docu(code, year):
    sales = dart.finstate(code, year)
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액 
    c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values[0] # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values[0]

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year) #'사외이사' / 전체 임원 수   
    out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    exec = exec.shape[0]

    employee = dart.report(code, '직원', year)['rgllbr_co'].to_list() # 총 직원 수
    
    output = {'sales': c_sales, 
              'pre_sales': p_sales, 
              'cash_div': c_div, 
              'pre_cash_civ': p_div,
              'stakeholder' : ratio_stakeholder,
              'num_outexecutives': out_exec,
              'num_executives': exec,
              'num_employee': employee}
    
    return output

In [ ]:
data = []
good_index = []
bad_index = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data.append(sales_docu(code, year))
        print(f'{index}, Good!')
        good_index.append(index)
    except:
        print(f'{index}, bad...')
        bad_index.append(index)
print(len(good_index), len(bad_index)) # 

# data downloading
with open ('s_docu.txt', 'w') as f:
    for x in df.iloc[good_index, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_docu.txt', 'w') as f:
    for x in df.iloc[bad_index, 0].values:
        f.write(f'{x}\n')
    f.close()

In [ ]:
api_key = '4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)
code, year = '311690', 2020

def report_docu(code, year):
    lst = dart.list(code, start= str(year), end=str(year+1))
    report_num = lst[lst['report_nm'].str.contains('사업보고서')].iloc[0, 5] # 해당년도 사업 보고서가 없는 경우
    (report_name, report_num) = lst[lst['report_nm'].str.contains('사업보고서')][['report_nm', 'rcept_no']].iloc[0].items()
    print(report_name[1])
    index = report_num[1]
    url = dart.sub_docs(index) #사업보고서
    i = url[url['title'].str.contains('연결재무제표')].index[0]
    html = url.loc[i, 'url']

    docu = pd.read_html(html)
    if docu[1].empty | docu[3].empty:
        print('Some document cannot be extracted')
    else:
        fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

        asset = fin_state[fin_state['tag'] == '자산총계'].values # 원래는 iloc[0, 1] # 자산 총계
        debt= fin_state[fin_state['tag'] == '부채총계'].values # 원래는 iloc[0, 1] # 부채 총계
        intangible = fin_state[fin_state['tag'] == '무형자산'].values # 원래는 iloc[0, 1] # 무형자산 

        income = docu[3].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서

        profit = income[income['tag'].str.contains('영업이익')].values # 원래는 iloc[0, 1] # ()는 마이너스를 뜻함 # 영업이익(손실)
        c_profit_tax = income[income['tag'].str.contains('법인세비용차감전')].values # 원래는 iloc[0, 1] # t기 법인세비용차감전순이익(손실)
        p_profit_tax = income[income['tag'].str.contains('법인세비용차감전')].values # t-1기 법인세비용차감전순이익(손실)
        
        dict_2 = {'asset': asset,
                    'debt': debt,
                    'intang': intangible,
                    'profit': profit,
                    'profit_without_tax': c_profit_tax,
                    'pre_profit_without_tax': p_profit_tax}   
    return dict_2                                                                                                          

# print(report_docu(code, year))

In [ ]:
data_re = []
good_index_re = []
bad_index_re = []
year = 2020
for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data_re.append(report_docu(code, year))
        print(f'{index}, Good!')
        good_index_re.append(index)
    except:
        print(f'{index}, bad...')
        bad_index_re.append(index)

In [ ]:
succees_report_docu = df.iloc[good_index_re, 0]
fail_report_docu = df.iloc[bad_index_re, 0]

# data downloading
with open ('s_report.txt', 'w') as f:
    for x in df.iloc[good_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_report.txt', 'w') as f:
    for x in df.iloc[bad_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
import pickle
with open('data_report.pickle', 'wb') as f:
    # for  data in data_re
    pickle.dump(data_re, f)

In [ ]:
with open('data_report.pickle', 'rb') as f:
    loaded = pickle.load(f)
print(loaded)
print(len(loaded))